Import Required Libraries

In [3]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, LSTM, Dense


Load tokenized tweets and sentiments

In [4]:
data = pd.read_csv('../data/processed/tweet_embeddings.csv')
# Convert embeddings from JSON strings back to numpy arrays
data['embedding'] = data['embedding'].apply(lambda x: np.array(json.loads(x)))
X = np.array(data['embedding'].tolist())  # Assuming embeddings are stored as lists
y = np.array((data['target']==4).astype(int).tolist())  # Binary target labels (0 or 1)

Split into training, validation, and testing sets

In [5]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


Load the pre-trained Word2Vec model using the gensim library and create an embedding matrix

In [6]:
from gensim.models import Word2Vec

# Load Word2Vec model
word2vec_model = Word2Vec.load('../models/word2vec/word2vec.model')

# Get the Word2Vec embedding dimension
embedding_dim = word2vec_model.vector_size
print("Embedding Dimension", embedding_dim)
# Create an embedding matrix
vocab_size = len(word2vec_model.wv.key_to_index) + 1
print("Total words", vocab_size)
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, index in word2vec_model.wv.key_to_index.items():
    embedding_matrix[index] = word2vec_model.wv[word]

Embedding Dimension 300
Total words 55794


Define the model using your pre-trained Word2Vec embeddings

In [7]:
# Define the LSTM model
model = Sequential()

# Embedding layer
embedding_layer = Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=X_train.shape[1],  # Number of tokens per tweet
    trainable=False  # Freeze the embeddings
)
model.add(embedding_layer)

# Add LSTM and Dense layers
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.build(input_shape=(None, X_train.shape[1]))  # Batch size is flexible

# Display model summary
model.summary()

d:\Hussam_ATOUI\github-hmatoui\twitter-sentiment-analysis\venv\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 300, 300)       │    16,738,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 300, 300)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │       160,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,898,701 (64.46 MB)

 Trainable params: 160,501 (626.96 KB)

 Non-trainable params: 16,738,200 (63.85 MB)

Compile and Train

In [8]:
from tqdm.keras import TqdmCallback
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    verbose=0,  # Turn off default verbosity
    callbacks=[TqdmCallback(verbose=1)]
)


  0%|          | 0/10 [00:00<?, ?epoch/s]

KeyboardInterrupt: 

Evaluate the Model

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Generate classification report
from sklearn.metrics import classification_report

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))


Save the Model

In [ ]:
model.save('../models/sentiment/sentiment_model.h5')
model.save('../app/model/sentiment_model.h5')